In [1]:
!pip install langChain
!pip install openai
!pip install PyPDF2
!pip install faiss_cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.7 MB/s eta 0:00:00


In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [3]:
import os
os.environ["OPENAI_API_KEY"]="sk-dmWO8PnfXmEXeKT3BlbkFJulCdllxl0zcCMeI1BTBA"

In [4]:
#PROVIDE THE PATH OF PDF FILE.
pdfreader=PdfReader('/content/SSRN_R1.pdf')

In [5]:
from typing_extensions import Concatenate
#read text from pdf
raw_text=""
for i,page in enumerate(pdfreader.pages):
  content=page.extract_text()
  if content:
    raw_text += content

In [6]:
raw_text

'See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/370653602\nGenerative AI\nPreprint  · May 2023\nCITATIONS\n0READS\n6,160\n4 author s, including:\nSome o f the author s of this public ation ar e also w orking on these r elat ed pr ojects:\nDesign Theories  View pr oject\nBusiness Analytics and R eal-time Pr ocess Analytics  View pr oject\nJochen Hartmann\nTechnische Univ ersität München\n24 PUBLICA TIONS \xa0\xa0\xa0484 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nPatrick Zschech\nFriedrich-Ale xander -Univ ersity of Erlang en-Nürnber g\n66 PUBLICA TIONS \xa0\xa0\xa0969 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Patrick Zschech  on 31 July 2023.\nThe user has r equest ed enhanc ement of the do wnlo aded file.Generative AI\nStefan Feuerriegel1,∗and Jochen Hartmann2and Christian Janiesch3and Patrick Zschech4\n1LMU Munich & Munich Center for Machine Learning, Geschwister-Scholl-Pl

In [7]:
#WE NEED TO SPLIT THE TEXT USING CHARATER TEXT SPLITTER SUCH THAT IT SHOULD NOT INCREASE TOKEN SIZE
text_splitter=CharacterTextSplitter(
    separator= "\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts=text_splitter.split_text(raw_text)

In [8]:
len(texts)

141

In [9]:
#download embiddings from Open AI
embeddings=OpenAIEmbeddings()

In [10]:
document_search=FAISS.from_texts(texts,embeddings)

In [11]:
document_search

In [12]:

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [13]:
chain=load_qa_chain(OpenAI(),chain_type="stuff")

In [14]:
query="give the summary of the pdf"
docs=document_search.similarity_search(query)
chain.run(input_documents=docs,question=query)

' This PDF discusses generative AI and how it is used in tasks such as image generation. It explores two types of generative AI models: diffusion probability models and reinforcement learning from human feedback (RLHF). RLHF is a three-step process which creates demonstration data for prompts, has users rank the quality of different outputs for a prompt, and learns a policy that generates desirable output via reinforcement learning.'